In [ ]:
import pandas as pd
import numpy as np
import pymongo
import matplotlib.pyplot as plt
import darksky_soccer as ds
import scipy.stats as scs

%load_ext autoreload
%autoreload 2

### Pull match data from sqlite database and combine with darksky weather data

In [ ]:
darksky_key = ds.get_api_key('/Users/bmcgarry/.secret/darksky.txt')
SEASON = 2011

match_stats = ds.match_rain_data('database.sqlite', SEASON, darksky_key)

### Create aggregate match stats with either pandas or sqlite

In [ ]:
#Aggregate with Pandas
team_stats_pandas = ds.calculate_aggregate_stats_pandas(match_stats)

#Aggregate with SQLite
team_stats_sql = ds.calculate_aggregate_stats_sqlite(match_stats, 'database.sqlite')

### Insert data into atlas instance

In [ ]:
atlas_user = 'brian_m'
atlas_key = ds.get_api_key('/Users/bmcgarry/.secret/atlas.txt')
cluster_name = 'chocolate-clusters-so6jw'
collection_name = 'soccer_stats'

In [ ]:
ds.insert_to_atlas(atlas_user,
                   atlas_key,
                   cluster_name,
                   collection_name,
                   team_stats_pandas)

### Pull match data from atlas

In [ ]:
url = f'mongodb+srv://{atlas_user}:{atlas_key}@{cluster_name}.mongodb.net/test?retryWrites=true&w=majority'
client = pymongo.MongoClient(url)
db = client.test
collection = db[collection_name]

df = pd.DataFrame(list(collection.find({'Season': SEASON})))
df.head()

### Plot Rain Win Percentages

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
df.sort_values('rain_win_pct', ascending=False, inplace=True)
ax.bar(df['Club'], df['rain_win_pct'])
ax.tick_params(axis='x', rotation=70)
ax.set_title(f'Win percentage In the Rain during the {SEASON} Season')
ax.set_ylabel('Win Percentage')
ax.set_xlabel('Teams');

### Does the English Premier League have more goals per game than in Bundesliga in the 2011/12 season?

In [ ]:
season_df = ds.get_season_data('database.sqlite', SEASON)
season_df['both_team_goals'] = season_df['FTHG'] + season_df['FTAG']
season_df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

for division in season_df['Div'].unique():
    goal_counts = season_df[season_df['Div'] == division]['both_team_goals']
    ax.hist(goal_counts, bins=10, alpha=.4, label=division)
    print(f'Mean Total Goals Scored per Game in {division}: {goal_counts.mean():.2f}');
    
ax.legend()
ax.set_title(f'Distribution of Total Goals Scored in the \nEnglish Premier League and Bundesliga for the {SEASON} Season')
ax.set_ylabel('Number of Games')
ax.set_xlabel('Number of Goals')
fig.tight_layout()

### Lets do a T-Test

Null Hypothesis: The total goals scored per game in the Premier League in the 2011/2012 season are the same or fewer than in Bundesliga

Alternative Hypothesis: The total goals scored per game in the Premier League in the 2011/2012 season are greater than in Bundesliga

I will set my alpha value to 0.1.


In [ ]:
premier_goals = season_df[season_df['Div'] == 'E0']['both_team_goals']
bundesliga_goals = season_df[season_df['Div'] == 'D1']['both_team_goals']

In [ ]:
scs.ttest_ind(premier_goals, bundesliga_goals, equal_var=False)